###  Report for Advanced Lane finding

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

#### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the third code cell of the IPython notebook located in "./examples/example.ipynb" (or in lines # through # of the file called `Advaced_Lane_P4.ipynb`).  

I used OpenCV functions findChessboardCorners and calibrateCamera here. A number of images of a chessboard, taken from different angles with the same camera, comprise the input. Arrays of object points, corresponding to the location (essentially indices) of internal corners of a chessboard, and image points, the pixel locations of the internal chessboard corners determined by findChessboardCorners, are fed to calibrateCamera which returns camera calibration and distortion coefficients. These can then be used by the OpenCV undistort function to undo the effects of distortion on any image produced by the same camera. 

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 


![image](./report/calibrated.JPG)

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

The image below depicts the results of applying undistort to one of the project images:

![Fig](./report/road_calibrated.JPG)

The difference can be noted by focusing on the hood of the car.

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

I performed a combination of sobel thresholding and HLS thresholding (S to be specific). There were many iterations were done with the threshold value for sobel as well as HLS. FOllowing is the results obtained. 


![Img2](./report/threshold.jpg)

There was no need to apply a region of interest on the sobel operator. This is becuase in the pipeline, perspective would change the image from dash view to topdown view. I tried applying the region of interest, results were the same and why wouldn't they be!

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform is a function named 'perspective_warped' in the Jupyter notebook, in the sixth code cells from the top. The function takes as inputs an image (img), as well as source (src) and destination (dst) points. I chose to hardcode the source and destination points in the following manner:


```python
src = src = np.float32([[580,460],[710,460],[1150,720],[150,720]])
dst = np.float32([[offset1, offset3], 
                      [img_size[0]-offset1, offset3], 
                      [img_size[0]-offset1, img_size[1]-offset2], 
                      [offset1, img_size[1]-offset2]])
```
I had considered programmatically determining source and destination points, but I felt that I would get better results carefully selecting points using one of the straight_lines test images for reference and assuming that the camera position will remain constant and that the road in the videos will remain relatively flat. The image below demonstrates the results of the perspective transform:

![Img2](./report/road_persp.jpg)![Img](./report/perspective.jpg)

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

I used a combination of 'find_lanes' and 'fit_lines' function in my provided jupyter notebook which identify lane lines and fit a second order polynomial to both right and left lane lines respectively.

The first of these computes a histogram of the bottom half of the image and finds the bottom-most x position (or "base") of the left and right lane lines. These locations were identified from the local maxima of the left and right halves of the histogram.  The function then identifies nine windows from which to identify lane pixels, each one centered on the midpoint of the pixels from the window below. This effectively maps the lane lines up to the top of the binary image, and speeds processing by only searching for activated pixels over a small portion of the image. Pixels belonging to each lane line are identified and the Numpy polyfit() method fits a second order polynomial to each set of pixels. Below is the image representing all the aforementioned processes: 


![Img2](./report/fit.jpg)

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

The radius of curvature is based upon this website and calculated in the code cell titled "Radius of Curvature and Distance from Lane Center Calculation" using this line of code (altered for clarity):
```python
curve_radius = ((1 + (2*fit[0]*y_0*x_meters_per_pixel + fit[1])**2)**1.5) / np.absolute(2*fit[0])
```

In this example, fit[0] is the first coefficient (the y-squared coefficient) of the second order polynomial fit, and fit[1] is the second (y) coefficient. y_0 is the y position within the image upon which the curvature calculation is based (the bottom-most y - the position of the car in the image - was chosen). x_meters_per_pixel is the factor used for converting from pixels to meters. This conversion was also used to generate a new fit with coefficients in terms of meters.

The position of the vehicle with respect to the center of the lane is calculated with the following lines of code:
```python
lane_center_position = (r_lane + l_lane) /2
position = image_shape[1]/2
center_dist = (position - lane_center_position) * x_meters_per_pix```
r_lane and l_lane are the x-intercepts of the right and left fits, respectively.  The car position is the difference between these intercept points and the image midpoint (assuming that the camera is mounted at the center of the vehicle). In my code, these ae two separate functions named 'find_curvature' and 'find_position'.

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

A 'draw_poly' funtion was created to plot the results back into road such that lane area could be identified. Minv was evaluated while taking the perspective transform. This Minv was def back into cv2.warpPerspective with warped image as the input. Once the input was obtained. This was merged with the actual untempered iamge using cv2.addWeighted with proper weights. 

Text was also added to this image in the font ```cv2.FONT_HERSHEY_SIMPLEX``` using ```cv2.putText```. This text was the output of ```find_radius``` and ```find_position```. Following is the image:

![Img](./report/plotted.jpg)

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./white.mp4)

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Issues: 
Finding the appropriate source points for perspective transform. This changed a lot of things for the effectiveness of the algorithm.
Lane line were not detected in shadows or dark light (challenge video). I worked on the challange video butstill facing issues with loosing the track of lanes near the tunnel. May be averaging and better custom binary thresholding would solve this.
What I have learned is that although it is relatively easy to finetune a software pipeline to work well for consistent road and weather conditions, it is a challenge finding a single combination which produces the same quality result in any condition.

For the third harder challenge video, since there are very small radius curves frequently, decreasing the window length of the perspctive transfowm would help. Also, there is a problem with detecting the edge line and not the tree lines. May be more finetuning of perpective source cordinated needeed.

Here's a [link to my challenge video result](./Untitled Folder/challenge_white.mp4)

This is not a very good result, and the shortcomings can be clearly seen. These were produced using Advaced_Lane_P4_harder.ipynb notebook which is just more tuning of the basic pipeline.

Here's a [link to my harder challenge video result](./Untitled Folder/challenge_harder_challenge_white.mp4)

Again, the algorith is doing the basic process but is getting distracted by frequent turns and tree baseline on the right side.
These were produced using Advaced_Lane_P4_harder_challenge.ipynb notebook which is just more tuning of the basic pipeline.